In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from time import perf_counter
import os
import seaborn as sns
from PIL import Image
from PIL import ImageEnhance
from skimage.io import imread
import matplotlib.pyplot as plt
import os, random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, Conv2D
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dense, Dropout
K.clear_session()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the dataset in Google Drive
data_dir = '/content/drive/MyDrive/tomato/train'


# Create the training dataset
'''training_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
'''
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255, #This rescales the pixel values of the images from a range of [0, 255] to [0, 1].
    #This normalization step helps improve the training process.
    rotation_range=40,
    horizontal_flip=True,
    fill_mode='nearest'
)
training_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(250, 250),
    batch_size=100,
    class_mode='categorical'  # Ensure this matches your loss function
)

Found 9987 images belonging to 10 classes.


In [ ]:
# Path to the dataset in Google Drive
data_dir_val = '/content/drive/MyDrive/tomato/val'

# Create the training dataset
'''validation_data = tf.keras.preprocessing.image_dataset_from_directory(data_dir_val,
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
'''
valid_datagen = ImageDataGenerator(rescale=1./255)

validation_data = valid_datagen.flow_from_directory(
    data_dir_val,
    target_size=(250, 250),
    batch_size=100,
    class_mode='categorical'  # Ensure this matches your loss function
)

Found 854 images belonging to 10 classes.


In [ ]:
'''target_names = training_data.class_names
print(target_names)'''
target_names = list(training_data.class_indices.keys())
print(target_names)

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(250, 250, 3), kernel_size=(11, 11), strides=(4, 4), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11, 11), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(BatchNormalization())

# Flattening
model.add(Flatten())

# 1st Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

# Output Layer
model.add(Dense(10))
model.add(Activation('softmax'))

# Summarize the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 60, 60, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 60, 60, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 30, 30, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 30, 30, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 20, 20, 256)         │       2,973,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 20, 20, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 10, 10, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 384)           │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 8, 8, 384)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 384)           │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 6, 6, 384)           │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 6, 6, 384)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 384)           │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 4, 4, 256)           │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 2, 2, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │              

 Total params: 27,165,450 (103.63 MB)

 Trainable params: 27,146,314 (103.55 MB)

 Non-trainable params: 19,136 (74.75 KB)

In [ ]:

#model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf

# Configure TensorFlow to use the GPU with dynamic memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')

        # Allow memory growth for the GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)

        # Or set a limit on GPU memory usage (if you want to limit usage)
        # tf.config.experimental.set_virtual_device_configuration(
        #     gpus[0],
        #     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

    except RuntimeError as e:
        print(e)

# Fit the model
model.fit(
    training_data,
    validation_data=validation_data,
    epochs=10
)


Physical devices cannot be modified after being initialized
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3872s 36s/step - accuracy: 0.4561 - loss: 1.8033 - val_accuracy: 0.1171 - val_loss: 3.3288
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.6870 - loss: 0.9963 - val_accuracy: 0.1522 - val_loss: 3.8405
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.7655 - loss: 0.7307 - val_accuracy: 0.1628 - val_loss: 4.4945
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.7977 - loss: 0.6121 - val_accuracy: 0.3033 - val_loss: 3.2693
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.8349 - loss: 0.4977 - val_accuracy: 0.3232 - val_loss: 3.1883
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.8432 - loss: 0.4649 - val_accuracy: 0.6265 - val_loss: 1.1803
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.8755 - loss: 0.3661 - val_accuracy: 0.7927 - val_loss: 0.6106
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━

In [ ]:
model.save("AlexNetModel1.hdf5")

In [ ]:
import imageio
import tensorflow
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model
from PIL import Image
target_names = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']
def run(source = None):
    model = tensorflow.keras.models.load_model('/content/AlexNetModel1.hdf5')
    img = imageio.imread(source)
    # Resize the image to a larger size that is compatible with the model
    img = Image.fromarray(img).resize((250, 250))  # Match the input shape of the model
    x = img_to_array(img)
    x = np.expand_dims(img, axis=0)
    x = x/255 #STANDARDIZATION
    prediction = model.predict(x)
    print("Predicted Image is:",target_names[np.argmax(prediction)])
run(source=r'/content/drive/MyDrive/tomato/val/Tomato___Target_Spot/0a51617c-ecb5-42ce-9d34-300307be3465___Com.G_TgS_FL 0976.JPG')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step
Predicted Image is: Tomato___Target_Spot
